In [ ]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path

os.chdir(Path(os.getcwd()).parent.parent.parent)
os.getcwd()

In [ ]:
import sys
sys.path.append('/home/tamir/workspace/automotive/src/python')

In [ ]:
import pandas as pd

from services.lanes.src.corridor import LanesCorridor, LanesCorridorParams

### Run On Set

In [ ]:
def run_on_set(set_path: Path, skip=1, max=-1, plot=False):
    b2b_pred_path = set_path.joinpath('results/infer/b2b_det.tsv')
    cametra_interface_data_path = set_path
    images_path = set_path.joinpath('images')

    pred_df = pd.read_csv(b2b_pred_path, sep='\t')
    max = len(pred_df.name.unique()[::skip]) if max == -1 else max
    relevant_imgs = pred_df.name.unique()[::skip][:max]
    pred_df = pred_df[pred_df.name.isin(relevant_imgs)]

    params = LanesCorridorParams(cametra_data_path=cametra_interface_data_path, 
                                  images_path=images_path,
                                  plot=plot)
    rbs = LanesCorridor(params=params)

    results_df = rbs.apply_all(detections=pred_df, 
                               save_path=b2b_pred_path.parent)
    
    return results_df
    

In [ ]:
run_on_set(Path('/home/tamir/s3_sync/eu_410/64b4ca8d628d0605eb5a0f2b'), skip=1, plot=True)

In [ ]:
run_on_set(Path('/home/tamir/s3_sync/europe_run_3_10/650fe9117edfb609dc1b8c8a'), skip=5, max=100)

In [ ]:
run_on_set(Path('/home/tamir/s3_sync/eu_410/64bcbd6e628d0605eb5a2dfc'), plot=True, skip=5, max=100) 

In [ ]:
results_df = pd.read_csv(Path('/home/tamir/workspace/eu_410_b2b_results_with_rbs.tsv'), sep='\t')
results_df.shape

In [ ]:
from services.lanes.src import statistics

statistics.calculate_lanes_statistics(detections=results_df)

In [ ]:
path_to_gt = Path('/home/tamir/s3_sync/eu_410/64bcbd6e628d0605eb5a2dfc/results/infer').joinpath('results_df_with_rbs.tsv')
path_to_prod = Path('/home/tamir/s3_sync/eu_410/64bcbd6e628d0605eb5a2dfc/results/infer').joinpath('results_df_with_rbs_prod.tsv')

gt_df = pd.read_csv(path_to_gt, sep='\t')
prod_df = pd.read_csv(path_to_prod, sep='\t')

In [ ]:
gt_df.loc[gt_df.is_outside_boundaries == 'OUTSIDE_BOUNDARIES', 'label'] = 'OUTSIDE_BOUNDARIES'
prod_df.loc[prod_df.is_outside_boundaries == 'OUTSIDE_BOUNDARIES', 'label'] = 'OUTSIDE_BOUNDARIES'

In [ ]:
# save the dataframes with the new labels
gt_df.to_csv(path_to_gt.parent.joinpath('results_df_with_rbs_changed.tsv'), sep='\t', index=False)
prod_df.to_csv(path_to_prod.parent.joinpath('results_df_with_rbs_prod_changed.tsv'), sep='\t', index=False)


In [ ]:
prod_df = pd.read_csv('/home/tamir/Downloads/superset_prod_results_with_rbs (2).tsv', sep='\t')
b2b_df = pd.read_csv('/home/tamir/Downloads/superset_b2b_results_with_rbs (1).tsv', sep='\t')
prod_df.shape, b2b_df.shape


In [ ]:
names = list(set.intersection(set(prod_df.name.unique()), set(b2b_df.name.unique())))[:5000]

prod_df_5k = prod_df[prod_df.name.isin(names)]
b2b_df_5k = b2b_df[b2b_df.name.isin(names)]

prod_df_5k.shape, b2b_df_5k.name.unique().shape, b2b_df_5k.shape, prod_df_5k.name.unique().shape

In [ ]:
# save the dataframes with 5k
# gt_df.to_csv(path_to_gt.parent.joinpath('results_df_with_rbs_changed.tsv'), sep='\t', index=False)
prod_df_5k.to_csv('/home/tamir/Downloads/prod_df_5k.tsv', sep='\t', index=False)
b2b_df_5k.to_csv('/home/tamir/Downloads/b2b_df_5k.tsv', sep='\t', index=False)


In [ ]:
prod_df = pd.read_csv('/home/tamir/Downloads/prod_df_5k.tsv', sep='\t')
b2b_df = pd.read_csv('/home/tamir/Downloads/b2b_df_5k.tsv', sep='\t')

prod_df.name.unique().shape, b2b_df.name.unique().shape

In [ ]:
prod_df.name.values

In [ ]:
def count_images(directory):
    dir_to_count = {}
    for root, _, files in os.walk(directory):
        if 'images' in root:
            dir_to_count[root] = len(files)
    return dir_to_count

# Specify the directory and file names
directory_path = '/home/tamir/s3_sync/eu_410'

# Call the function to find the file paths
result = count_images(directory_path)
result

In [ ]:
import os

def find_file_paths(directory, file_names):
    file_paths = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file in file_names:
                file_paths.append(os.path.join(root, file))
    return file_paths

# Specify the directory and file names
directory_path = '/home/tamir/s3_sync/eu_410'

# Call the function to find the file paths
result = find_file_paths(directory_path, prod_df.name.unique())

# Print the result
for file_path in result:
    print(file_path)

In [ ]:
import numpy as np

# b2b_df[b2b_df.is_outside_boundaries == 'INSIDE_BOUNDARIES'].label.unique()
pd.Series([v.split('_')[0] + '_outside_boundary' for v in prod_df[prod_df.is_outside_boundaries == 'OUTSIDE_BOUNDARIES'].label.values]).shape
lbls = pd.Series([v.split('_')[0] + '_outside_boundary' for v in prod_df[prod_df.is_outside_boundaries == 'OUTSIDE_BOUNDARIES'].label.values]).values
lbls.shape


In [ ]:
prod_df.loc[prod_df.is_outside_boundaries == 'OUTSIDE_BOUNDARIES', 'label'] = lbls
prod_df.label.unique()

In [ ]:
prod_df[prod_df.is_outside_boundaries == 'INSIDE_BOUNDARIES'].label.unique()

In [ ]:
prod_df.to_csv('/home/tamir/Downloads/superset_prod_results_with_rbs (2).tsv', sep='\t', index=False)

In [ ]:
prod_df = pd.read_csv('/home/tamir/Downloads/prod_df_5k.tsv', sep='\t')
prod_df.shape


In [ ]:
prod_df.label.unique()

In [ ]:
pd.Series([v.split('_')[0] for v in prod_df[prod_df.is_outside_boundaries == 'OUTSIDE_BOUNDARIES'].label.values]).unique()

In [ ]:
b2b_df = pd.read_csv('/home/tamir/Downloads/b2b_df_5k.tsv', sep='\t')
b2b_df.shape

In [ ]:
c = ['name', 'x_center', 'y_center', 'width', 'height', 'label',
                           'score', 'is_occluded', 'is_truncated', 'd3_separation', 'l_label',
                           'r_label', 'is_rider_on_2_wheels']
b2b_df[c].head()

In [ ]:
prod_df.label.unique()

In [ ]:
'1692223060999783.png' in prod_df.name.values
'1695052008999853.png' in prod_df.name.values

## Visualize Evaluations

## Visualize Evaluations

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df_rbs = pd.read_csv('/home/tamir/Downloads/eval_table_results_max_recall_test.tsv', sep='\t')
df = pd.read_csv('/home/tamir/Desktop/eval_table_results_max_recall_test.tsv', sep='\t')

In [ ]:
df.head()

In [ ]:
def calc_recall(x):
    a = x[1:].split('/')
    if int(a[1]) == 0:
        return 0
    return int(a[0]) / int(a[1])

df['evaluated_recall'] = df.recall.apply(calc_recall)
df_rbs['evaluated_recall'] = df_rbs.recall.apply(calc_recall)

In [ ]:
# Assuming df and df_rbs are your DataFrames
df_4w = df[df.class_name == '4w']
df_rbs_4w = df_rbs[df_rbs.class_name == '4w']

# Set up the subplot
plt.figure(figsize=(10, 6))

# Set the width of the bars
bar_width = 0.3

# Set positions for the bars
positions_df_4w = np.arange(len(df_4w))
positions_df_rbs_4w = np.arange(len(df_rbs_4w)) + bar_width + 0.1  # Adding a gap of 0.1

# Plot for df_4w in blue
plt.barh(positions_df_4w, df_4w['evaluated_recall'], height=bar_width, color='orange', label='df_4w')

# Plot for df_rbs_4w in orange
plt.barh(positions_df_rbs_4w, df_rbs_4w['evaluated_recall'], height=bar_width, color='green', label='df_rbs_4w')

# Add labels on the start of each bar
for i, value in enumerate(df_4w['evaluated_recall']):
    plt.text(value, positions_df_4w[i], f'{value:.2f}', ha='right', va='center')

for i, value in enumerate(df_rbs_4w['evaluated_recall']):
    plt.text(value, positions_df_rbs_4w[i], f'{value:.2f}', ha='right', va='center')

# Set y-axis ticks and labels
plt.yticks(positions_df_4w + bar_width / 2, df_4w.bin)

# Set labels and title
plt.xlabel('Evaluated Recall')
plt.ylabel('Bins')
plt.title('Comparison of df_4w and df_rbs_4w')
plt.legend()  # Display legend

# Adjust layout for better spacing
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
# Assuming df and df_rbs are your DataFrames
df_4w = df[df.class_name == '4w']
df_rbs_4w = df_rbs[df_rbs.class_name == '4w']

# Set up the subplot
plt.figure(figsize=(10, 6))

# Set the width of the bars
bar_width = 0.3

# Set positions for the bars
positions_df_4w = np.arange(len(df_4w))
positions_df_rbs_4w = np.arange(len(df_rbs_4w)) + bar_width + 0.1  # Adding a gap of 0.1

# Plot for df_4w in blue
plt.barh(positions_df_4w, df_4w['precision_loose'], height=bar_width, color='orange', label='df_4w')

# Plot for df_rbs_4w in orange
plt.barh(positions_df_rbs_4w, df_rbs_4w['precision_loose'], height=bar_width, color='green', label='df_rbs_4w')

# Add labels on the start of each bar
for i, value in enumerate(df_4w['precision_loose']):
    plt.text(value, positions_df_4w[i], f'{value:.2f}', ha='right', va='center')

for i, value in enumerate(df_rbs_4w['precision_loose']):
    plt.text(value, positions_df_rbs_4w[i], f'{value:.2f}', ha='right', va='center')

# Set y-axis ticks and labels
plt.yticks(positions_df_4w + bar_width / 2, df_4w.bin)

# Set labels and title
plt.xlabel('Precision Loose')
plt.ylabel('Bins')
plt.title('Comparison of df_4w and df_rbs_4w')
plt.legend()  # Display legend

# Adjust layout for better spacing
plt.tight_layout()

# Display the plot
plt.show()